Get images and save them independently as 2D slices

Dependencies

In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons

import scipy

import SimpleITK as sitk
reader = sitk.ImageFileReader()
reader.SetImageIO("MetaImageIO")

import numpy as np

import os

import pathlib

from natsort import natsorted

#Set GPU/Cuda Device to run model on
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

np.random.seed(46)

Using cuda device


Dataset Directories

In [2]:
#Toy Dataset Directory from Spider | Grand Challenge 
dummy_train_img_dir = pathlib.Path(r"spider_toy_dset/dummy_train_images")
dummy_train_label_dir = pathlib.Path(r"spider_toy_dset/dummy_train_labels")
dummy_test_img_dir = pathlib.Path(r"spider_toy_dset/dummy_test_images")
dummy_test_label_dir= pathlib.Path(r"spider_toy_dset/dummy_test_labels")

Image Class, Returns 3D Array

In [3]:
from transforms import mri_transforms


#TODO add bool image label for interp 
class Mri:
    def __init__(self, path):
        mri_mha = sitk.ReadImage(path, imageIO = "MetaImageIO") #explicitly setting ioreader just in case
        
       #TODO use sitk permute to change dimensions to 2 0 1 to help with going through slices for interpolating in 2d
        if(mri_mha.GetSize()[0] > mri_mha.GetSize()[1] or mri_mha.GetSize()[0] > mri_mha.GetSize()[2]): #if z axis isn't first
          mri_mha = sitk.PermuteAxes(mri_mha, (2, 0, 1))
          
        #get 3d array from mri
        mri_a = np.array(sitk.GetArrayFromImage(mri_mha)) #mri_array

        #transpose array to format z x y or z y x 
        if(mri_a.shape[0] > mri_a.shape[1] or mri_a.shape[0] > mri_a.shape[2]): #if z axis isn't first
          mri_a = np.transpose(mri_a, (2, 0, 1))

        '''
        if(mri_a.shape[2] > mri_a.shape[1]): #bring images to vertical orientation
          mri_a = np.transpose(mri_a, (0, 2, 1))
        '''
      
        mri_a_float32 = mri_a.astype(dtype = np.float32)
        #TODO: set bounds to [-1000, 2000] https://en.wikipedia.org/wiki/Hounsfield_scale
        self.hu_a = mri_a_float32